<a href="https://colab.research.google.com/github/sri-dhurkesh/Hindi_character_recognition/blob/main/Hindi_character_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cloning the github repository**

In [4]:
!git clone https://github.com/sri-dhurkesh/Hindi_character_recognition.git

Cloning into 'Hindi_character_recognition'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.


In [ ]:
print('Hello World')

Hello World


In [1]:
print('Yaswanthan')

Yaswanthan
